In [9]:
import scipy
import numpy as np

### Calculate n-Sphere survace area
Survace of the sphere embedded in n-dimentional space is: 
$$
S_{n-1}(R) = \frac{2\,\pi^\frac{n}{2}}{\Gamma\left(\frac{n}{2}\right)}R^{n-1}
$$
we have R=1
$$
\log S_{n-1}(1) = \log(2) + \frac{n}{2}\log\pi - \log \Gamma\left(\frac{n}{2}\right)
$$

In [2]:
n=512
log_S = np.log(2) + (n/2)*np.log(np.pi) - scipy.special.loggamma(n/2)
log_S

-867.9681031603942

In [3]:
from omegaconf import OmegaConf
from hydra.utils import instantiate
from evaluation.ijb_evals import instantiate_list
from evaluation.face_recognition_test import Face_Fecognition_test

cfg = OmegaConf.load('/app/configs/uncertainty_benchmark/ijb_evals.yaml')

# instantiate
open_set_identification_metrics = instantiate_list(
    cfg.open_set_identification_metrics
)
open_set_uncertainty_metrics = instantiate_list(cfg.open_set_uncertainty_metrics)
closed_set_identification_metrics = instantiate_list(
    cfg.closed_set_identification_metrics
)
verification_metrics = instantiate_list(cfg.verification_metrics)

test_dataset = instantiate(cfg.test_dataset)

methods = []
method_types = []
if "open_set_identification_methods" in cfg:
    methods += cfg.open_set_identification_methods
    method_types += ["open_set_identification"] * len(
        cfg.open_set_identification_methods
    )
for method, method_type in zip(methods, method_types):
    evaluation_function = instantiate(method.evaluation_function)
    template_pooling = instantiate(method.template_pooling_strategy)
    tt = Face_Fecognition_test(
        evaluation_function=evaluation_function,
        test_dataset=test_dataset,
        embeddings_path=method.embeddings_path,
        template_pooling_strategy=template_pooling,
        use_detector_score=method.use_detector_score,
        use_two_galleries=cfg.use_two_galleries,
        recompute_template_pooling=cfg.recompute_template_pooling,
        open_set_identification_metrics=open_set_identification_metrics,
        closed_set_identification_metrics=closed_set_identification_metrics,
        verification_metrics=verification_metrics,
        open_set_uncertainty_metrics=open_set_uncertainty_metrics,
    )

>>>> Reload from backup: /app/datasets/arcface_ijb/IJBC/backup.npz ...
>>>> Reload from backup: /app/datasets/arcface_ijb/IJBC/gallery_prob_backup.npz ...
>>>> Reload embeddings from: /app/datasets/arcface_ijb/IJBC/embeddings/scf_embs_IJBC.npz
>>>> process_embeddings: Norm False, Detect_score True, Flip False
Using detection score normalization


In [5]:
(
    g1_templates_feature,
    g1_template_unc,
    g1_unique_ids,
) = tt.get_template_subsets(
    tt.test_dataset.g1_templates, tt.test_dataset.g1_ids
)
print("g1_templates_feature:", g1_templates_feature.shape)  # (1772, 512)

(
    probe_templates_feature,
    probe_template_unc,
    probe_unique_ids,
) = tt.get_template_subsets(tt.test_dataset.probe_templates, tt.test_dataset.probe_ids)
print("probe_templates_feature:", probe_templates_feature.shape)  # (19593, 512)

g1_templates_feature: (1772, 512)
probe_templates_feature: (19593, 512)


In [28]:
pos_id = 1
neg_id = 28936
pos_idx = np.where(probe_unique_ids == pos_id)[0][0]
neq_idx = np.where(probe_unique_ids == neg_id)[0][0]
positive_cosine_sim = probe_templates_feature[[pos_idx],:] @ g1_templates_feature.T
negative_cosine_sim = probe_templates_feature[[neq_idx],:] @ g1_templates_feature.T

In [31]:
np.max(positive_cosine_sim), np.max(negative_cosine_sim)

(0.695818168081169, 0.26441042146959054)

In [37]:
from scipy.special import ive, hyp0f1

In [42]:
hyp0f1(256, (100**2)/4)

14600.889158420143

In [84]:
from scipy.special import ive, hyp0f1, loggamma
def compute_log_z_prob(similarities, kappa, beta=0.5, n=256):
    K = similarities.shape[1]
    print(ive(n - 1, kappa))
    log_iv = np.log(ive(n - 1, kappa)) + kappa
    log_c = (n-1)*np.log(kappa) - n*np.log(2*np.pi) - log_iv
    logit_sum = np.sum(np.exp(similarities * kappa), axis=1)*(1 - beta) / K
    #print(f'Logit sum: {logit_sum}')
    alpha_value = hyp0f1(n, kappa**2 / 4)
    #print(f'Alpha value: {alpha_value}')
    log_z_prob = log_c + np.log(logit_sum + alpha_value*beta)
    #print(f'Log z prob: {log_z_prob}')
    return log_z_prob

def compute_uniform_aposteriory(similarities, kappa, beta=0.5, n=256):
    # compute log z prob
    log_z_prob = compute_log_z_prob(similarities, kappa, beta, n)
    print(f'Log z prob: {log_z_prob}')
    log_uniform_dencity = loggamma(n) - np.log(2) - n*np.log(np.pi)
    print(f'Log uniform dencity: {log_uniform_dencity}')
    log_beta = np.log(beta)
    print(f'Log beta : {log_beta}')
    log_prob = log_uniform_dencity + log_beta - log_z_prob
    print(f'Log uniform prob: {log_prob}')
    return log_prob


In [85]:
loggamma(5)

3.1780538303479458

In [90]:
kappa = 50
beta = 0.5
n = probe_templates_feature.shape[1] / 2

compute_uniform_aposteriory(positive_cosine_sim, kappa, beta, n)
#compute_uniform_aposteriory(negative_cosine_sim, kappa, beta, n)

1.950466804074554e-169
Log z prob: [892.15604646]
Log uniform dencity: 867.9681031603943
Log beta : -0.6931471805599453
Log uniform prob: [-24.88109048]


array([-24.88109048])

In [80]:
kappa = 1000
beta = 0.5
n = probe_templates_feature.shape[1] / 2

compute_uniform_aposteriory(positive_cosine_sim, kappa, beta, n)
#compute_uniform_aposteriory(negative_cosine_sim, kappa, beta, n)

1.1199021188562076e-16
Log z prob: [1015.35434411]
Log uniform dencity: 867.9681031603943
Log beta : -0.6931471805599453
Log uniform prob: [-148.07938813]


array([-148.07938813])